In [2]:
import sys
import os

# Get the current working directory
# Get project root (folder above 'notebooks')
project_root = os.path.dirname(os.getcwd())

# Path to scripts folder
scripts_path = os.path.join(project_root, "scripts")

# Add to sys.path
sys.path.append(project_root)
sys.path.append(scripts_path)
# Add db folder
db_path = os.path.join(project_root, "db")
sys.path.append(db_path)
import pandas as pd

In [3]:
from connection import engine, SessionLocal
from config import  DATA_PATHS
from reviews import insert_reviews_from_df

In [3]:
df = pd.read_sql("SELECT * FROM banks;", engine)

2025-12-01 12:37:58,063 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-12-01 12:37:58,065 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 12:37:58,067 INFO sqlalchemy.engine.Engine select current_schema()
2025-12-01 12:37:58,068 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 12:37:58,070 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-12-01 12:37:58,071 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 12:37:58,079 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-01 12:37:58,080 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [4]:
df.head()

,bank_id,bank_name,app_name
0,1,CBE,Commercial Bank of Ethiopia
1,2,BoAMobile,BoA Mobile
2,3,DashenBank,Dashen Bank


In [5]:
sentiment_df = pd.read_csv('../data/processed/sentiment_only_data.csv')

In [6]:
sentiment_df.head()

,review,rating,date,bank,source,normalized_review,compound_score,sentiment_label
0,CBE ይለያል።,5,2025-11-29,CBE,com.combanketh.mobilebanking,cbe,0.0000,NEUTRAL
1,it's special for me,5,2025-11-29,CBE,com.combanketh.mobilebanking,special,0.4019,POSITIVE
2,Make it user friendly.,2,2025-11-29,CBE,com.combanketh.mobilebanking,make user friendly,0.4939,POSITIVE
3,maaliif daddafee install gaafata,3,2025-11-28,CBE,com.combanketh.mobilebanking,maaliif daddafee install gaafata,0.0000,NEUTRAL
4,good app,5,2025-11-28,CBE,com.combanketh.mobilebanking,good app,0.4404,POSITIVE


#### Insert cleaned review data using Python (psycopg2 or SQLAlchemy recommended)


In [8]:
insert_reviews_from_df(sentiment_df)

2025-12-01 12:38:27,295 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-12-01 12:38:27,296 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 12:38:27,298 INFO sqlalchemy.engine.Engine select current_schema()
2025-12-01 12:38:27,299 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 12:38:27,300 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-12-01 12:38:27,301 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 12:38:27,303 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-01 12:38:27,306 INFO sqlalchemy.engine.Engine SELECT bank_id FROM banks WHERE bank_name = %(bank_name)s LIMIT 1
2025-12-01 12:38:27,308 INFO sqlalchemy.engine.Engine [generated in 0.00153s] {'bank_name': 'CBE'}
2025-12-01 12:38:27,315 INFO sqlalchemy.engine.Engine 
                INSERT INTO reviews
                (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
                VALUES
                (%(bank_id)s, %(review_text)s,

In [6]:
all_review = pd.read_sql("SELECT * FROM reviews;", engine)
all_review.head()

2025-12-01 13:24:26,793 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-01 13:24:26,794 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-12-01 13:24:26,795 INFO sqlalchemy.engine.Engine [cached since 190.9s ago] {'table_name': 'SELECT * FROM reviews;', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-12-01 13:24:26,798 INFO sqlalchemy.engine.Engine SELECT * FROM reviews;
2025-12-01 13:24:26,800 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 13:24:26,813 INFO sqlalchemy.engine.Engine ROLLBACK


,review_id,bank_id,review_text,rating,review_date,sentiment_label,sentiment_score,source
0,12034,1,CBE ይለያል።,5,2025-11-29,NEUTRAL,0.00,com.combanketh.mobilebanking
1,12035,1,it's special for me,5,2025-11-29,POSITIVE,0.40,com.combanketh.mobilebanking
2,12036,1,Make it user friendly.,2,2025-11-29,POSITIVE,0.49,com.combanketh.mobilebanking
3,12037,1,maaliif daddafee install gaafata,3,2025-11-28,NEUTRAL,0.00,com.combanketh.mobilebanking
4,12038,1,good app,5,2025-11-28,POSITIVE,0.44,com.combanketh.mobilebanking


#### ● Write SQL queries to verify data integrity (e.g., count reviews per bank, average rating)

##### Count total reviews per bank

In [4]:
count_per_bank_sql = """ 
SELECT b.bank_name, COUNT(r.review_id) AS total_reviews
FROM banks b
LEFT JOIN reviews r ON b.bank_id = r.bank_id
GROUP BY b.bank_name
ORDER BY total_reviews DESC;
"""


count_per_bank = pd.read_sql(count_per_bank_sql, engine)
count_per_bank.head()

2025-12-01 13:21:15,907 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-12-01 13:21:15,908 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 13:21:15,911 INFO sqlalchemy.engine.Engine select current_schema()
2025-12-01 13:21:15,912 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 13:21:15,916 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-12-01 13:21:15,917 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-01 13:21:15,924 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-01 13:21:15,925 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

,bank_name,total_reviews
0,BoAMobile,400
1,CBE,400
2,DashenBank,400


#### Average rating per bank

In [5]:
averate_rate_per_bank_sql = """ 
SELECT b.bank_name, ROUND(AVG(r.rating), 2) AS avg_rating
FROM banks b
LEFT JOIN reviews r ON b.bank_id = r.bank_id
GROUP BY b.bank_name
ORDER BY avg_rating DESC;
"""


average_rate_per_bank = pd.read_sql(averate_rate_per_bank_sql, engine)
average_rate_per_bank.head()

2025-12-01 13:22:36,297 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-01 13:22:36,298 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-12-01 13:22:36,299 INFO sqlalchemy.engine.Engine [cached since 80.38s ago] {'table_name': ' \nSELECT b.bank_name, ROUND(AVG(r.rating), 2) AS avg_rating\nFROM banks b\nLEFT JOIN reviews r ON b.bank_id = r.bank_id\nGROUP BY b.bank_name\nORDER BY avg_rating DESC;\n', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-12-01 13:22:36,301 INFO sqlalchemy.engine.Engine  
SELECT b.b

,bank_name,avg_rating
0,CBE,4.13
1,DashenBank,3.96
2,BoAMobile,3.38
